In [ ]:
%tensorflow_version 1.x

In [ ]:
import os
import random
import scipy.io
import keras
import tensorflow as tf
from google.colab.patches import cv2_imshow
import numpy as np
from glob import glob
from google.colab import drive
import matplotlib.pyplot as plt

from keras.layers import Conv2DTranspose, ConvLSTM2D, BatchNormalization, TimeDistributed, Conv2D, Flatten, Dense, MaxPooling2D, Input
from keras.models import Sequential, load_model

drive.mount('/content/gdrive')   

In [ ]:
FUSION2020_FOLDER_PATH = "/content/gdrive/My Drive/FUSION2020"
NUMBER_OF_CLASSES = 36
NUMBER_OF_EXAMPLES_FOR_EACH_CLASS = 60
NUMBER_OF_EXAMPLES_USED_FOR_TRAINING_AND_VALIDATION_NETWORK = 10
NUMBER_OF_EXAMPLES_USED_FOR_TRAINING_AND_VALIDATION_PERCEPTRON = 5


main_working_folder = FUSION2020_FOLDER_PATH
input_tactil_matrix_path = os.path.join(main_working_folder, 'LSTMtactil.mat')
input_arduino_matrix_path = os.path.join(main_working_folder, 'LSTMKinesthetic.mat')
output_results_matrix_path = os.path.join(main_working_folder, 'Perceptron.mat')

print(input_tactil_matrix_path)
print(input_arduino_matrix_path)
print(output_results_matrix_path)

In [12]:
# Function to define and train the classification network model.

def train_model(training_data, training_labels):
  seq = Sequential()
  # # # # #
  seq.add(Dense(64, activation='relu'))
  seq.add(Dense(64, activation='relu'))
  # # # # #
  seq.add(Dense(NUMBER_OF_CLASSES, activation = "softmax"))

  seq.compile(optimizer=keras.optimizers.Adam(lr=0.0001),
              loss="categorical_crossentropy",
              metrics=['accuracy'])
  history = seq.fit(training_data, training_labels, batch_size = 16, epochs=200, validation_split = 0.2, shuffle = True)

  print(seq.summary())
  return seq, history

In [13]:
# Function to change matrix index.

def change_matrix_index(matrix):

  new_matrix = -1*np.ones(shape=(matrix.shape[1], matrix.shape[0]))
  for index in range(matrix.shape[1]):
    new_matrix[index,:] = matrix[:,index]

  return new_matrix


#Function to load training and evaluation data.

def load_training_and_evaluation_data(matrix_path, number_of_examples_to_get_from_each_class = NUMBER_OF_EXAMPLES_USED_FOR_TRAINING_AND_VALIDATION_PERCEPTRON, number_of_examples_by_class_in_input_matrix = (NUMBER_OF_EXAMPLES_FOR_EACH_CLASS-NUMBER_OF_EXAMPLES_USED_FOR_TRAINING_AND_VALIDATION_NETWORK)):

  matrix = scipy.io.loadmat(matrix_path)                                        # We load the .mat as a matrix (numpy array).
  matrix = change_matrix_index(matrix["output"])

  training_data_matrix = np.zeros(shape=(number_of_examples_to_get_from_each_class*NUMBER_OF_CLASSES, NUMBER_OF_CLASSES))
  number_of_evaluation_examples_for_each_class = (number_of_examples_by_class_in_input_matrix - number_of_examples_to_get_from_each_class)
  evaluation_data_matrix = np.zeros(shape=(number_of_evaluation_examples_for_each_class*NUMBER_OF_CLASSES, NUMBER_OF_CLASSES))

  index = 0
  training_data_labels_list = []
  evaluation_data_labels_list = []

  for class_index in range(NUMBER_OF_CLASSES):
    output_matrix_index = index*number_of_examples_to_get_from_each_class
    output_matrix_offset = output_matrix_index+number_of_examples_to_get_from_each_class
    evaluation_output_matrix_index = index*number_of_evaluation_examples_for_each_class
    evaluation_output_matrix_offset = evaluation_output_matrix_index+number_of_evaluation_examples_for_each_class
    input_matrix_index = index*number_of_examples_by_class_in_input_matrix
    input_matrix_offset = input_matrix_index+number_of_examples_to_get_from_each_class

    training_data_matrix[output_matrix_index:output_matrix_offset,:] = matrix[input_matrix_index:input_matrix_offset,:]

    evaluation_data_matrix[evaluation_output_matrix_index:evaluation_output_matrix_offset, :] = matrix[input_matrix_offset:input_matrix_offset+number_of_evaluation_examples_for_each_class, :]

    training_data_labels_list = training_data_labels_list + [class_index]*number_of_examples_to_get_from_each_class
    evaluation_data_labels_list = evaluation_data_labels_list + [class_index]*number_of_evaluation_examples_for_each_class

    index += 1

  return training_data_matrix, training_data_labels_list, evaluation_data_matrix, evaluation_data_labels_list

In [ ]:

tactils_training_data, tactils_training_labels, tactils_evaluation_data, tactils_evaluation_labels = load_training_and_evaluation_data(input_tactil_matrix_path, number_of_examples_to_get_from_each_class=NUMBER_OF_EXAMPLES_USED_FOR_TRAINING_AND_VALIDATION_PERCEPTRON, number_of_examples_by_class_in_input_matrix = (NUMBER_OF_EXAMPLES_FOR_EACH_CLASS-NUMBER_OF_EXAMPLES_USED_FOR_TRAINING_AND_VALIDATION_NETWORK))
arduino_training_data, arduino_training_labels, arduino_evaluation_data, arduiono_evaluation_labels = load_training_and_evaluation_data(input_arduino_matrix_path, number_of_examples_to_get_from_each_class=NUMBER_OF_EXAMPLES_USED_FOR_TRAINING_AND_VALIDATION_PERCEPTRON, number_of_examples_by_class_in_input_matrix = (NUMBER_OF_EXAMPLES_FOR_EACH_CLASS-NUMBER_OF_EXAMPLES_USED_FOR_TRAINING_AND_VALIDATION_NETWORK))

print("Loaded data shapes:")
print("Arduino training data")
print(arduino_training_data.shape)
print("Arduino training labels")
print(len(arduino_training_labels))
print("Tactils training data")
print(tactils_training_data.shape)
print("Arduino training labels")
print(len(tactils_training_labels))
print("Arduino evaluation data")
print(arduino_evaluation_data.shape)
print("Arduino evaluation labels")
print(len(arduiono_evaluation_labels))
print("Tactils evaluation data")
print(tactils_evaluation_data.shape)
print("Tactils evaluation labels")
print(len(tactils_evaluation_labels))

combined_data = np.zeros(shape=((arduino_training_data.shape[0], 2*NUMBER_OF_CLASSES)))
combined_data[:,0:NUMBER_OF_CLASSES] = arduino_training_data
combined_data[:,NUMBER_OF_CLASSES:] = tactils_training_data
labels = list(keras.utils.to_categorical(np.array(tactils_training_labels)))

combined_data = list(combined_data)
zip_list = list(zip(combined_data, labels))
random.shuffle(zip_list)
combined_data, labels = zip(*zip_list)
combined_data = np.array(combined_data)
labels = np.array(labels)

print("Combined data shapes:")
print(combined_data.shape)
print(labels.shape)

model, history = train_model(combined_data, labels)

combined_evaluation_data = np.zeros(shape=((arduino_evaluation_data.shape[0], 2*NUMBER_OF_CLASSES)))
combined_evaluation_data[:, 0:NUMBER_OF_CLASSES] = arduino_evaluation_data
combined_evaluation_data[:, NUMBER_OF_CLASSES:] = tactils_evaluation_data
evaluation_labels = list(keras.utils.to_categorical(np.array(tactils_evaluation_labels)))
combined_evaluation_data = np.array(combined_evaluation_data)
evaluation_labels = np.array(evaluation_labels)

print("Evaluation combined data shapes:")
print(combined_evaluation_data.shape)
print(evaluation_labels.shape)

score = model.evaluate(combined_evaluation_data, evaluation_labels)
print('Test loss: {} / Test accuracy: {}'.format(score[0], score[1]))

output_file = os.path.join(main_working_folder,'evaluation_perceptron.txt')

with open(output_file, 'w') as f:
  print('Test loss: {} / Test accuracy: {}'.format(score[0], score[1]), file=f)

evaluation_predictions = model.predict(combined_evaluation_data)

print(evaluation_predictions.shape)

output_matrix = -1 * np.ones((evaluation_predictions.shape[1],evaluation_predictions.shape[0]))

for i in range(evaluation_predictions.shape[0]):
  output_matrix[:,i] = evaluation_predictions[i]

scipy.io.savemat(output_results_matrix_path, {'output':output_matrix})

In [ ]:
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.savefig(os.path.join(FUSION2020_FOLDER_PATH,"review/model_accuracy_Perceptron_review_{}_examples_used_to_train_originals_{}_examples_used_to_train_perceptron.png".format(NUMBER_OF_EXAMPLES_USED_FOR_TRAINING_AND_VALIDATION_NETWORK, NUMBER_OF_EXAMPLES_USED_FOR_TRAINING_AND_VALIDATION_PERCEPTRON)))
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.savefig(os.path.join(FUSION2020_FOLDER_PATH,"review/model_loss_Perceptron_review_{}_examples_used_to_train_originals_{}_examples_used_to_train_perceptron.png".format(NUMBER_OF_EXAMPLES_USED_FOR_TRAINING_AND_VALIDATION_NETWORK, NUMBER_OF_EXAMPLES_USED_FOR_TRAINING_AND_VALIDATION_PERCEPTRON)))
plt.show()